This Notebook will present the results of the Exploratory Data Analysis

# Clone Repository

In [1]:
# Executes the cell in bash mode
%%bash

if [ ! -d "/content/deep-deblurring/" ]; 
    then 
        git clone https://github.com/ElPapi42/deep-deblurring;
        cd deep-deblurring/
    else 
        cd deep-deblurring/; 
        git pull; 
fi;

git checkout development
cd ..

Branch 'book_save' set up to track remote branch 'book_save' from 'origin'.


Cloning into 'deep-deblurring'...
Switched to a new branch 'book_save'


# Install Requirements

In [0]:
# Executes the cell in bash mode
%%bash

#pip uninstall -y tensorflow tensor2tensor tensorboard tensorboardcolab tensorflow-datasets tensorflow-estimator tensorflow-gan tensorflow-hub tensorflow-metadata tensorflow-privacy tensorflow-probability

pip install colab-env
pip install --upgrade grpcio

#pip install -r "/content/deep-deblurring/model/requirements.txt"

cd deep-deblurring/model/
python setup.py install
cd /content/

In [0]:
# Add deblurrer package to path

import sys
sys.path += ['/content/deep-deblurring/model']

# Data Download

In [5]:
# Setup credentials from google drive

import colab_env

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
from deblurrer.scripts.datasets import  compile_all

compile_all.run('/content/deep-deblurring/model/datasets')

  0%|          | 0.00/1.49G [00:00<?, ?B/s]

100%|██████████| 1.49G/1.49G [00:42<00:00, 38.1MB/s]



Generating .csv files
.csv files successfully generated
Generating TFRecords
Generating train.csv TFRecords
Generating valid.csv TFRecords
Generating test.csv TFRecords
TFRecords succesfully generated


# Hyper-Parameters

In [0]:
import os

os.environ['EPOCHS'] = '1'
os.environ['BATCH_SIZE'] = '4'
os.environ['IMAGE_SIZE'] = '128'
os.environ['IMAGE_SIZE_MULTIPLIER'] = '2'
os.environ['FPN_CHANNELS'] = '128'
os.environ['DISC_FILTERS'] = '6'
os.environ['DISC_CONV_COUNT'] = '2'
os.environ['GEN_LR'] = '0.00001'
os.environ['DISC_LR'] = '0.00001'
os.environ['USE_MIXED_PRECISION'] = '0'

# Training

In [10]:
# Starts training

from deblurrer.scripts.training import train

model, gen_opt, disc_opt, strategy = train.run(
    '/content/deep-deblurring/model/datasets/tfrecords',
    output_folder='/content/deep-deblurring/model/output',
    warm_epochs=0,
)

80142336/80134624 [==============================] - 3s 0us/step
Model: "deblur_gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fpn_generator (FPNGenerator) multiple                  5236175   
_________________________________________________________________
double_scale_discriminator ( multiple                  6973590   
_________________________________________________________________
model_1 (Model)              (None, None, None, 256)   1735488   
Total params: 13,945,253
Trainable params: 13,908,887
Non-trainable params: 36,366
_________________________________________________________________
Starting model training...
132/132 [==============================] - 35s 263ms/step - gen_loss: 0.2486 - disc_loss: 4.3435 - val_gen_loss: 0.1187 - val_disc_loss: 4.0852


In [0]:
model, gen_opt, disc_opt, strategy = train.run(
    '/content/deep-deblurring/datasets/tfrecords',
    model,
    gen_opt,
    disc_opt,
    strategy,
    output_folder='/content/deep-deblurring/output'
)